# NetworkConfigParser

A small module to parse structured documents, like Cisco or Juniper network device configurations. Maintains
relationships among lines for ease of further parsing and analysis. Parses IP addresses for easier matching using the
ipaddress library.

## Quick Start and Examples

---

### Example 1: Find references to an interface name

1. We will read in the example configuration shown below, stored in the `config` variable as a string. The `parse_from_str()` function will parse this string, returning a list of `DocumentLine()` objects.

In [1]:
from search_helpers import *
from parser import *
import ipaddress as ipa

config = """interface TenGigE0/1/0/1
 description Backbone Circuit to North Pudsey from Metric Networks
 cdp
 mtu 2060
 ipv4 address 192.0.2.101 255.255.255.252
 load-interval 30
!
interface Loopback10
 description Router ID
 ipv4 address 192.0.2.1 255.255.255.255
!
router isis IGP
 net 49.0000.1920.0000.2001.00
 log adjacency changes
 address-family ipv4 unicast
  metric-style wide
  mpls traffic-eng level-1-2
  mpls traffic-eng router-id Loopback10
 !
 interface Loopback10
  passive
  circuit-type level-2-only
  address-family ipv4 unicast
  !
 !
 interface TenGigE0/1/0/1
  circuit-type level-2-only
  point-to-point
  address-family ipv4 unicast
   metric 1000
   mpls ldp sync
!
rsvp
 interface TenGigE0/1/0/1
 !
 interface TenGigE0/0/0/0
 !
!
mpls traffic-eng
 interface TenGigE0/0/0/0
 !
 interface TenGigE0/1/0/1
 !
!
mpls ldp
 !
 igp sync delay on-session-up 10
 router-id 192.0.2.1
 !
 session protection
 !
 interface TenGigE0/0/0/0
 !
 interface TenGigE0/1/0/1
 !
router static
 0.0.0.0/0 192.0.2.102
 203.0.113.100/30 192.0.2.102
"""

doc_lines = parse_from_str(config)

doc_lines[0:6]

[<DocumentLine gen=1 num_children=5 line_num=1: "interface TenGigE0/1/0/1">,
 <DocumentLine gen=2 num_children=0 line_num=2: " description Backbone Circuit to North Pudsey from Metric Networks">,
 <DocumentLine gen=2 num_children=0 line_num=3: " cdp">,
 <DocumentLine gen=2 num_children=0 line_num=4: " mtu 2060">,
 <DocumentLine gen=2 num_children=0 line_num=5: " ipv4 address 192.0.2.101 255.255.255.252">,
 <DocumentLine gen=2 num_children=0 line_num=6: " load-interval 30">]

Next, let's identify lines pertaining to TenGigE0/1/0/1. The `find_lines()` function helps us do this.

We supply the list created in the step above, plus a search term to identify the lines we want matched. If the search term is a string, that term is searched as a regular expression (using `re.search`).

In [22]:
find_lines(doc_lines, r'TenGigE0/1/0/1')

[<DocumentLine gen=1 num_children=5 line_num=1: "interface TenGigE0/1/0/1">,
 <DocumentLine gen=2 num_children=3 line_num=26: " interface TenGigE0/1/0/1">,
 <DocumentLine gen=2 num_children=0 line_num=34: " interface TenGigE0/1/0/1">,
 <DocumentLine gen=2 num_children=0 line_num=42: " interface TenGigE0/1/0/1">,
 <DocumentLine gen=2 num_children=0 line_num=54: " interface TenGigE0/1/0/1">]


By default, only the matched lines are returned. Parent lines are not included by default, but in this case we will want to see the section in which the interface appears. So we add `include_ancestors=True` to get the interface name.

In [23]:
find_lines(doc_lines, r'TenGigE0/1/0/1', include_ancestors=True)

[<DocumentLine gen=1 num_children=5 line_num=1: "interface TenGigE0/1/0/1">,
 <DocumentLine gen=1 num_children=7 line_num=12: "router isis IGP">,
 <DocumentLine gen=2 num_children=3 line_num=26: " interface TenGigE0/1/0/1">,
 <DocumentLine gen=1 num_children=4 line_num=33: "rsvp">,
 <DocumentLine gen=2 num_children=0 line_num=34: " interface TenGigE0/1/0/1">,
 <DocumentLine gen=1 num_children=4 line_num=39: "mpls traffic-eng">,
 <DocumentLine gen=2 num_children=0 line_num=42: " interface TenGigE0/1/0/1">,
 <DocumentLine gen=1 num_children=10 line_num=45: "mpls ldp">,
 <DocumentLine gen=2 num_children=0 line_num=54: " interface TenGigE0/1/0/1">]

The filtered list contains the matched interface lines with all section header lines ("router isis", "mpls ldp", etc.) leading to those matches.

We can also include immediate children of the matches:

In [24]:
find_lines(doc_lines, r'TenGigE0/1/0/1', include_children=True)

[<DocumentLine gen=1 num_children=5 line_num=1: "interface TenGigE0/1/0/1">,
 <DocumentLine gen=2 num_children=0 line_num=2: " description Backbone Circuit to North Pudsey from Metric Networks">,
 <DocumentLine gen=2 num_children=0 line_num=3: " cdp">,
 <DocumentLine gen=2 num_children=0 line_num=4: " mtu 2060">,
 <DocumentLine gen=2 num_children=0 line_num=5: " ipv4 address 192.0.2.101 255.255.255.252">,
 <DocumentLine gen=2 num_children=0 line_num=6: " load-interval 30">,
 <DocumentLine gen=2 num_children=3 line_num=26: " interface TenGigE0/1/0/1">,
 <DocumentLine gen=3 num_children=0 line_num=27: "  circuit-type level-2-only">,
 <DocumentLine gen=3 num_children=0 line_num=28: "  point-to-point">,
 <DocumentLine gen=3 num_children=2 line_num=29: "  address-family ipv4 unicast">,
 <DocumentLine gen=2 num_children=0 line_num=34: " interface TenGigE0/1/0/1">,
 <DocumentLine gen=2 num_children=0 line_num=42: " interface TenGigE0/1/0/1">,
 <DocumentLine gen=2 num_children=0 line_num=54: "

And we can include all descendants of the matched lines as well:

In [26]:
find_lines(doc_lines, r'TenGigE0/1/0/1', include_all_descendants=True)

[<DocumentLine gen=1 num_children=5 line_num=1: "interface TenGigE0/1/0/1">,
 <DocumentLine gen=2 num_children=0 line_num=2: " description Backbone Circuit to North Pudsey from Metric Networks">,
 <DocumentLine gen=2 num_children=0 line_num=3: " cdp">,
 <DocumentLine gen=2 num_children=0 line_num=4: " mtu 2060">,
 <DocumentLine gen=2 num_children=0 line_num=5: " ipv4 address 192.0.2.101 255.255.255.252">,
 <DocumentLine gen=2 num_children=0 line_num=6: " load-interval 30">,
 <DocumentLine gen=2 num_children=3 line_num=26: " interface TenGigE0/1/0/1">,
 <DocumentLine gen=3 num_children=0 line_num=27: "  circuit-type level-2-only">,
 <DocumentLine gen=3 num_children=0 line_num=28: "  point-to-point">,
 <DocumentLine gen=3 num_children=2 line_num=29: "  address-family ipv4 unicast">,
 <DocumentLine gen=4 num_children=0 line_num=30: "   metric 1000">,
 <DocumentLine gen=4 num_children=0 line_num=31: "   mpls ldp sync">,
 <DocumentLine gen=2 num_children=0 line_num=34: " interface TenGigE0/

---

## Example: Controlling regex options

`find_lines()` permits setting flags in re.search. Set regex_flags=re.IGNORECASE to perform case-insensitive matches:

In [29]:
find_lines(doc_lines, r'INTERFACE', regex_flags=re.IGNORECASE)

[<DocumentLine gen=1 num_children=5 line_num=1: "interface TenGigE0/1/0/1">,
 <DocumentLine gen=1 num_children=2 line_num=8: "interface Loopback10">,
 <DocumentLine gen=2 num_children=4 line_num=20: " interface Loopback10">,
 <DocumentLine gen=2 num_children=3 line_num=26: " interface TenGigE0/1/0/1">,
 <DocumentLine gen=2 num_children=0 line_num=34: " interface TenGigE0/1/0/1">,
 <DocumentLine gen=2 num_children=0 line_num=36: " interface TenGigE0/0/0/0">,
 <DocumentLine gen=2 num_children=0 line_num=40: " interface TenGigE0/0/0/0">,
 <DocumentLine gen=2 num_children=0 line_num=42: " interface TenGigE0/1/0/1">,
 <DocumentLine gen=2 num_children=0 line_num=52: " interface TenGigE0/0/0/0">,
 <DocumentLine gen=2 num_children=0 line_num=54: " interface TenGigE0/1/0/1">]

---

## Example: Matching successive children

Let's extract interface metric settings from "router isis" above. We can do this easily by giving find_items() multiple search terms in the form of a list or a tuple.

We'll keep ancestors in the result for readability.

In [31]:
find_lines(doc_lines, ['interface', 'metric'], regex_flags=re.IGNORECASE, include_ancestors=True)

[<DocumentLine gen=1 num_children=5 line_num=1: "interface TenGigE0/1/0/1">,
 <DocumentLine gen=2 num_children=0 line_num=2: " description Backbone Circuit to North Pudsey from Metric Networks">,
 <DocumentLine gen=1 num_children=7 line_num=12: "router isis IGP">,
 <DocumentLine gen=2 num_children=3 line_num=26: " interface TenGigE0/1/0/1">,
 <DocumentLine gen=3 num_children=2 line_num=29: "  address-family ipv4 unicast">,
 <DocumentLine gen=4 num_children=0 line_num=30: "   metric 1000">]

Multiple search terms match anywhere in the hierarchy. The only requirement to fulfill the match
is that the second term be found in any descendant of the first match, the third term is found in
any descendant of the second match, and so on.

To perform stricter matching where the second match must be made in the immediate child of the
first match, add recurse_search=False:

In [33]:
find_lines(doc_lines, ['interface', 'metric'], regex_flags=re.IGNORECASE, include_ancestors=True, recurse_search=False)

[<DocumentLine gen=1 num_children=5 line_num=1: "interface TenGigE0/1/0/1">,
 <DocumentLine gen=2 num_children=0 line_num=2: " description Backbone Circuit to North Pudsey from Metric Networks">]

Now our IS-IS metric lines are missing. This makes sense, because 'interface' and 'metric' are separated in the IS-IS config with "address-family ipv4". Let's add that term in:

In [38]:
find_lines(doc_lines, ['interface', 'address-family', 'metric'], regex_flags=re.IGNORECASE, include_ancestors=True, recurse_search=False)

[<DocumentLine gen=1 num_children=7 line_num=12: "router isis IGP">,
 <DocumentLine gen=2 num_children=3 line_num=26: " interface TenGigE0/1/0/1">,
 <DocumentLine gen=3 num_children=2 line_num=29: "  address-family ipv4 unicast">,
 <DocumentLine gen=4 num_children=0 line_num=30: "   metric 1000">]

Now we see our result as expected.

---

## Example: Callback functions as search terms

Let's find interfaces matching the IP 192.0.2.1.

We start with searching interfaces:

In [43]:
intf_search_result = find_lines(doc_lines, ['interface', '192.0.2.1'], regex_flags=re.IGNORECASE, include_ancestors=True)
intf_search_result

[<DocumentLine gen=1 num_children=5 line_num=1: "interface TenGigE0/1/0/1">,
 <DocumentLine gen=2 num_children=0 line_num=5: " ipv4 address 192.0.2.101 255.255.255.252">,
 <DocumentLine gen=1 num_children=2 line_num=8: "interface Loopback10">,
 <DocumentLine gen=2 num_children=0 line_num=10: " ipv4 address 192.0.2.1 255.255.255.255">]

Our result includes Te0/1/0/1 that matches the IP in the regex, but is not the address we want.

DocumentLine objects parse anything that looks like an IP address or network, and stores it as an object from the excellent ipaddress module. Those are stored in the `ip_addrs` and `ip_nets` attributes respectively:

In [46]:
intf_search_result[1].ip_addrs, intf_search_result[3].ip_addrs

(frozenset({IPv4Address('192.0.2.101')}),
 frozenset({IPv4Address('192.0.2.1')}))

In [47]:
intf_search_result[1].ip_nets, intf_search_result[3].ip_nets

(frozenset({IPv4Network('192.0.2.100/30')}),
 frozenset({IPv4Network('192.0.2.1/32')}))

DocumentLine objects even have a method to match on IPv4 (or v6)Address and Network objects, `has_ip()`:

In [49]:
intf_search_result[1].has_ip(ipa.ip_address('192.0.2.1')), intf_search_result[3].has_ip(ipa.ip_address('192.0.2.1'))

(False, True)

Regular expressions can't use the has_ip method, because they search the entire line by default.

In addition to regular expressions, `find_lines()` can take a function as a search term. The function must take a DocumentLine as the first and only argument, and return a bool to indicate a successful match.

Lambda expressions, therefore, work very well here.

In [50]:
find_lines(doc_lines, ['interface', lambda x: x.has_ip(ipa.ip_address('192.0.2.1'))], regex_flags=re.IGNORECASE, include_ancestors=True)

[<DocumentLine gen=1 num_children=2 line_num=8: "interface Loopback10">,
 <DocumentLine gen=2 num_children=0 line_num=10: " ipv4 address 192.0.2.1 255.255.255.255">]

To find all /30 networks defined in the configuration:

In [56]:
find_lines(doc_lines, lambda x: any(True for i in x.ip_nets if i.prefixlen == 30), include_ancestors=True)

[<DocumentLine gen=1 num_children=5 line_num=1: "interface TenGigE0/1/0/1">,
 <DocumentLine gen=2 num_children=0 line_num=5: " ipv4 address 192.0.2.101 255.255.255.252">,
 <DocumentLine gen=1 num_children=2 line_num=56: "router static">,
 <DocumentLine gen=2 num_children=0 line_num=58: " 203.0.113.100/30 192.0.2.102">]

Find addresses that lie within a particular /30:

In [60]:
find_lines(doc_lines, lambda x: any(i in ipa.ip_network('192.0.2.100/30') for i in x.ip_addrs), include_ancestors=True)

[<DocumentLine gen=1 num_children=5 line_num=1: "interface TenGigE0/1/0/1">,
 <DocumentLine gen=2 num_children=0 line_num=5: " ipv4 address 192.0.2.101 255.255.255.252">,
 <DocumentLine gen=1 num_children=2 line_num=56: "router static">,
 <DocumentLine gen=2 num_children=0 line_num=57: " 0.0.0.0/0 192.0.2.102">,
 <DocumentLine gen=2 num_children=0 line_num=58: " 203.0.113.100/30 192.0.2.102">]

With callbacks, any attribute of DocumentLine can be chosen as a search term. The behavior of [CiscoConfParse's find_parent_objects()](https://www.pennington.net/py/ciscoconfparse2/api_CiscoConfParse.html#ciscoconfparse2.CiscoConfParse.find_parent_objects) can be replicated as follows:

In [78]:
find_lines(doc_lines, lambda x: re_search_dl('interface', x) and any(re_search_dl(r'metric \d+', c) for c in x.all_descendants), include_ancestors=True)

[<DocumentLine gen=1 num_children=7 line_num=12: "router isis IGP">,
 <DocumentLine gen=2 num_children=3 line_num=26: " interface TenGigE0/1/0/1">]

A concise helper method that generates the above function is included, `parent_child_cb()`. The above then becomes:

In [79]:
find_lines(doc_lines, parent_child_cb('interface', r'metric \d+'), include_ancestors=True)

[<DocumentLine gen=1 num_children=7 line_num=12: "router isis IGP">,
 <DocumentLine gen=2 num_children=3 line_num=26: " interface TenGigE0/1/0/1">]

Functions as search terms are exceptionally versatile.

---

## Example: Extracting all IP addresses and networks referenced in a configuration

IPs are easy to gather with a set comprehension (shown below) or a list comprehension.

In [10]:
all_ip_addrs = {j for i in doc_lines for j in i.ip_addrs if not i.is_comment}
all_ip_addrs

{IPv4Address('0.0.0.0'),
 IPv4Address('192.0.2.1'),
 IPv4Address('192.0.2.101'),
 IPv4Address('192.0.2.102'),
 IPv4Address('203.0.113.100')}

In [11]:
all_ip_networks = {j for i in doc_lines for j in i.ip_nets if not i.is_comment}
all_ip_networks

{IPv4Network('0.0.0.0/0'),
 IPv4Network('192.0.2.1/32'),
 IPv4Network('192.0.2.100/30'),
 IPv4Network('203.0.113.100/30')}

---

## Example: Converting output to a plain string

find_lines() has the convert_match parameter. It takes a function name which accepts a DocumentLine object and returns anything the user may want.

Instead of outputting DocumentLine objects for the match, it's easy to have find_lines() substitute the string instead:

In [19]:
find_lines(doc_lines, 'interface', convert_match=str, include_ancestors=True)

['interface TenGigE0/1/0/1',
 'interface Loopback10',
 <DocumentLine gen=1 num_children=7 line_num=12: "router isis IGP">,
 ' interface Loopback10',
 ' interface TenGigE0/1/0/1',
 <DocumentLine gen=1 num_children=4 line_num=33: "rsvp">,
 ' interface TenGigE0/1/0/1',
 ' interface TenGigE0/0/0/0',
 <DocumentLine gen=1 num_children=4 line_num=39: "mpls traffic-eng">,
 ' interface TenGigE0/0/0/0',
 ' interface TenGigE0/1/0/1',
 <DocumentLine gen=1 num_children=10 line_num=45: "mpls ldp">,
 ' interface TenGigE0/0/0/0',
 ' interface TenGigE0/1/0/1']

Note that the ancestor lines are not converted.

There are two ways to convert those as well. It's possible to use the convert_family parameter in the same way:

In [20]:
find_lines(doc_lines, 'interface', convert_match=str, convert_family=str, include_ancestors=True)


['interface TenGigE0/1/0/1',
 'interface Loopback10',
 'router isis IGP',
 ' interface Loopback10',
 ' interface TenGigE0/1/0/1',
 'rsvp',
 ' interface TenGigE0/1/0/1',
 ' interface TenGigE0/0/0/0',
 'mpls traffic-eng',
 ' interface TenGigE0/0/0/0',
 ' interface TenGigE0/1/0/1',
 'mpls ldp',
 ' interface TenGigE0/0/0/0',
 ' interface TenGigE0/1/0/1']

Or simply perform a list comprehension converting all results to str.

In [21]:
[str(i) for i in find_lines(doc_lines, 'interface', include_ancestors=True)]


['interface TenGigE0/1/0/1',
 'interface Loopback10',
 'router isis IGP',
 ' interface Loopback10',
 ' interface TenGigE0/1/0/1',
 'rsvp',
 ' interface TenGigE0/1/0/1',
 ' interface TenGigE0/0/0/0',
 'mpls traffic-eng',
 ' interface TenGigE0/0/0/0',
 ' interface TenGigE0/1/0/1',
 'mpls ldp',
 ' interface TenGigE0/0/0/0',
 ' interface TenGigE0/1/0/1']

---

## Example: Extracting IP addresses present in each section

Going back to our IP address example, let's see if we can get better detail. Let's gather the IP addresses and networks present in each section of the config.

In [24]:
def gather_ips(o):
    return {str(o): o.ip_addrs | o.ip_nets}
find_lines(doc_lines, lambda o: len(o.ip_addrs) or len(o.ip_nets), convert_match=gather_ips, convert_family=str, include_ancestors=True)

['interface TenGigE0/1/0/1',
 {' ipv4 address 192.0.2.101 255.255.255.252': frozenset({IPv4Network('192.0.2.100/30'),
             IPv4Address('192.0.2.101')})},
 'interface Loopback10',
 {' ipv4 address 192.0.2.1 255.255.255.255': frozenset({IPv4Address('192.0.2.1'),
             IPv4Network('192.0.2.1/32')})},
 'mpls ldp',
 {' router-id 192.0.2.1': frozenset({IPv4Address('192.0.2.1')})},
 'router static',
 {' 0.0.0.0/0 192.0.2.102': frozenset({IPv4Address('0.0.0.0'),
             IPv4Network('0.0.0.0/0'),
             IPv4Address('192.0.2.102')})},
 {' 203.0.113.100/30 192.0.2.102': frozenset({IPv4Address('192.0.2.102'),
             IPv4Address('203.0.113.100'),
             IPv4Network('203.0.113.100/30')})}]

As with functions providing match logic, convert_match and convert_family are also very powerful.

---

## Example: flatten_family set to False

In our example above, parsing that list is a bit unwieldy with its mixed string and dict representations. We can choose flatten_family=False to project each match as a list of lists:

In [25]:
def gather_ips(o):
    return {str(o): o.ip_addrs | o.ip_nets}
find_lines(doc_lines, lambda o: len(o.ip_addrs) or len(o.ip_nets), convert_match=gather_ips, convert_family=str, include_ancestors=True, flatten_family=False)

[['interface TenGigE0/1/0/1',
  {' ipv4 address 192.0.2.101 255.255.255.252': frozenset({IPv4Network('192.0.2.100/30'),
              IPv4Address('192.0.2.101')})}],
 ['interface Loopback10',
  {' ipv4 address 192.0.2.1 255.255.255.255': frozenset({IPv4Address('192.0.2.1'),
              IPv4Network('192.0.2.1/32')})}],
 ['mpls ldp', {' router-id 192.0.2.1': frozenset({IPv4Address('192.0.2.1')})}],
 ['router static',
  {' 0.0.0.0/0 192.0.2.102': frozenset({IPv4Address('0.0.0.0'),
              IPv4Network('0.0.0.0/0'),
              IPv4Address('192.0.2.102')})}],
 ['router static',
  {' 203.0.113.100/30 192.0.2.102': frozenset({IPv4Address('192.0.2.102'),
              IPv4Address('203.0.113.100'),
              IPv4Network('203.0.113.100/30')})}]]

Now, the matched line is always at the end of each sub-list, and the ancestor lines are the strings that precede the match.